In [1]:
import pickle
import rdkit
from rdkit import Chem
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from __future__ import absolute_import, division, print_function, unicode_literals

# standard python
import numpy as np
import scipy
#import pathlib

import os
# plotting, especially for jupyter notebooks
import matplotlib
#matplotlib.rcParams['text.usetex'] = True # breaks for some endpoint labels
from matplotlib import pyplot as plt
from IPython.display import Image
import pandas as pd
# tensorflow
import tensorflow as tf
#tf.enable_eager_execution() # needed for tf version 1 or it stages operations but does not do them
from tensorflow import keras
from tensorflow.keras import layers, regularizers
tf.keras.backend.clear_session()  # For easy reset of notebook state.
# local routines
#from chemdataprep import load_PDBs,load_countsfromPDB,load_diametersfromPDB,find_chemnames
#from toxmathandler import load_tscores

#checkpoint_path = "/home2/ajgreen4/Read-Across_w_GAN/Models/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

print("tensorflow version",tf.__version__,". Executing eagerly?",tf.executing_eagerly())
print("Number of GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

2025-11-30 12:51:51.868665: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 12:51:55.772875: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 12:52:02.638654: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


tensorflow version 2.20.0 . Executing eagerly? True
Number of GPUs:  0


W0000 00:00:1764525125.381816 1605803 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Load Data

In [2]:
# Load training data, val and test data
file_path = 'train.pkl'
with open(file_path, 'rb') as file:
    train_df = pickle.load(file)
#print(train_df)
# good fit = lower negative number
# more negative docking score = better fit 


# Load validation data
file_path = 'validation_small_enantiomers_stable_full_screen_docking_MOL_margin3_49878_10368_5184.pkl'
with open(file_path, 'rb') as file:
    val_df = pickle.load(file)
#print(val_df)


# Load test data
file_path = 'test_small_enantiomers_stable_full_screen_docking_MOL_margin3_50571_10368_5184.pkl'
with open(file_path, 'rb') as file:
    test_df = pickle.load(file)
#print(test_df)

# Get atoms in list

In [3]:
#train_df.head(20)

In [4]:

def atomic_number_to_symbol(atomic_number):
    periodic_table = {
        1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F',
        15: 'P', 16: 'S', 17: 'Cl', 35: 'Br', 53: 'I'
    }
    return periodic_table.get(atomic_number, 'Unknown')

# Define dataset paths
datasets = {
    'train': 'train.pkl',
    'val': 'validation_small_enantiomers_stable_full_screen_docking_MOL_margin3_49878_10368_5184.pkl',
    'test': 'test_small_enantiomers_stable_full_screen_docking_MOL_margin3_50571_10368_5184.pkl'
}

mollists = {}

for dataset_name, file_path in datasets.items():
    with open(file_path, 'rb') as file:
        df = pickle.load(file)
    
    mollist = []
    for mol in df['rdkit_mol_cistrans_stereo']:
        molecule = []
        if mol is not None and isinstance(mol, Chem.Mol):
            if mol.GetNumConformers() > 0:
                conf = mol.GetConformer(0)
                for atom in mol.GetAtoms():
                    atomic_number = atom.GetAtomicNum()
                    atom_type = atomic_number_to_symbol(atomic_number)
                    pos = conf.GetAtomPosition(atom.GetIdx())
                    coordinates = np.array([pos.x, pos.y, pos.z])
                    molecule.append((atom_type, coordinates))
        mollist.append(molecule)
    
    mollists[dataset_name] = mollist
    print(f"{dataset_name.capitalize()} Sample (first molecule):")
    print(mollist[0] if mollist else "No valid molecules found.\n")


Train Sample (first molecule):
[('Br', array([-2.04649997, -1.8743    ,  0.33809999])), ('C', array([1.38030005, 0.0871    , 0.14910001])), ('C', array([ 0.0986    , -0.0115    ,  0.83929998])), ('C', array([0.90759999, 1.29069996, 0.8251    ])), ('C', array([ 2.61890006, -0.6832    ,  0.1062    ])), ('C', array([ 1.89660001, -0.2475    , -1.17390001])), ('C', array([-1.22039998, -0.1279    ,  0.1161    ])), ('C', array([-1.46070004,  0.5363    , -1.21200001])), ('C', array([-2.17440009,  1.03050005,  0.012     ]))]
Val Sample (first molecule):
[('Br', array([-3.21449995, -0.0071    , -0.52410001])), ('N', array([0.5399    , 1.46490002, 0.80440003])), ('C', array([ 0.59130001, -1.55040002,  0.7726    ])), ('C', array([-0.83429998, -1.36479998,  0.2483    ])), ('C', array([ 1.75919998, -1.22930002, -0.1627    ])), ('C', array([-1.23389995, -0.0048    , -0.33129999])), ('C', array([ 1.72179997,  0.0517    , -0.99519998])), ('C', array([-0.85650003,  1.25820005,  0.44670001])), ('C', arra

In [5]:
# split into train, val, split to get views
train_mollist = mollists['train']
val_mollist = mollists['val']
test_mollist = mollists['test']

# Train test split

In [6]:
%%time
X_train = train_mollist    
y_train = train_df['top_score']

X_val = val_mollist             
y_val = val_df['top_score']    

X_test = test_mollist          
y_test = test_df['top_score']  

CPU times: user 267 μs, sys: 27 μs, total: 294 μs
Wall time: 301 μs


In [ ]:

def speciesmap(atom_type):
    atom_to_number = {
        'H': 1,   # Hydrogen
        'C': 6,   # Carbon
        'N': 7,   # Nitrogen
        'O': 8,   # Oxygen
        'F': 9,   # Fluorine
        'P': 15,  # Phosphorus
        'S': 16,  # Sulfur
        'Cl': 17, # Chlorine
        'Br': 35, # Bromine
        'I': 53   # Iodine
    }
    return np.array([atom_to_number.get(atom_type, 0)])  # Returns 0 if atom type is not recognized

In [11]:
# Load weights and views
ws_train = np.load('ws_train.npy')
vs_train = np.load('vs_train.npy')
ws_val = np.load('ws_val.npy')
vs_val = np.load('vs_val.npy')
ws_test = np.load('ws_test.npy')
vs_test = np.load('vs_test.npy')

# Load dimensions
Natoms_train = np.load('Natoms_train.npy')
Nviews_train = np.load('Nviews_train.npy')
Natoms_val = np.load('Natoms_val.npy')
Nviews_val = np.load('Nviews_val.npy')
Natoms_test = np.load('Natoms_test.npy')
Nviews_test = np.load('Nviews_test.npy')

chiral_train = [ws_train, vs_train]
chiral_val = [ws_val, vs_val]
chiral_test = [ws_test, vs_test]

In [12]:
print(ws_train.shape)
print(vs_train.shape)
print(ws_val.shape)
print(vs_val.shape)
print(ws_test.shape)
print(vs_test.shape)

(234622, 29)
(234622, 29, 116)
(49878, 29)
(49878, 29, 116)
(50571, 29)
(50571, 29, 116)


In [13]:
from qm7_transformercode import small_views
single_atom_train = small_views(vs_train)
single_atom_val = small_views(vs_val)
single_atom_test = small_views(vs_test)
print(single_atom_train.shape)
print(single_atom_val.shape)
print(single_atom_test.shape)

(234622, 29, 29, 4)
(49878, 29, 29, 4)
(50571, 29, 29, 4)


In [16]:
atom_embeddings_train = np.load("atom_embeddings_train.npy", mmap_mode='r')
atom_embeddings_val   = np.load("atom_embeddings_val.npy",   mmap_mode='r')
atom_embeddings_test  = np.load("atom_embeddings_test.npy",  mmap_mode='r')

print("Loaded atom embeddings!")



Loaded atom embeddings!


In [17]:
print(atom_embeddings_train.shape) # atomic number (1), 3d coordinates (3), single atom properties (8), padded zero (4)
print(atom_embeddings_val.shape)
print(atom_embeddings_test.shape)
print(atom_embeddings_train.dtype)
print(atom_embeddings_val.dtype)
print(atom_embeddings_test.dtype)

(234622, 29, 29, 20)
(49878, 29, 29, 20)
(50571, 29, 29, 20)
float64
float64
float64


In [18]:
atom_embeddings_train[0][0][0]

memmap([  35.   ,    0.   ,    0.   ,    0.   ,    2.96 ,   79.904,
           7.   ,   17.   ,  120.   , 1139.9  ,  324.6  ,   23.5  ,
           0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
           0.   ,    0.   ])

In [19]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Copy data
Xtr = atom_embeddings_train.copy()
Xval = atom_embeddings_val.copy()
Xte = atom_embeddings_test.copy()

# Create masks (nonzero atoms)
mask_tr = (Xtr[..., 0] != 0)
mask_val = (Xval[..., 0] != 0)
mask_te = (Xte[..., 0] != 0)

# Fit scaler ONLY on training atom properties
props_tr = Xtr[..., 4:12][mask_tr]   # assuming properties are columns 4–11 (8 features)
prop_scaler = StandardScaler().fit(props_tr)

# Apply transform to all three sets
Xtr[..., 4:12][mask_tr]  = prop_scaler.transform(Xtr[..., 4:12][mask_tr])
Xval[..., 4:12][mask_val] = prop_scaler.transform(Xval[..., 4:12][mask_val])
Xte[..., 4:12][mask_te]  = prop_scaler.transform(Xte[..., 4:12][mask_te])

# In[14]:

print(Xtr.shape)
print(Xval.shape)
print(Xte.shape)

(234622, 29, 29, 20)
(49878, 29, 29, 20)
(50571, 29, 29, 20)


In [20]:
Xtr[0][0][0]

array([35.        ,  0.        ,  0.        ,  0.        ,  0.72364863,
       21.91144044,  3.98654186,  2.5486692 ,  7.97021195, -0.13421879,
        3.37350309,  3.12961286,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [21]:

labelsG_train = y_train 
labelsG_val = y_val     
labelsG_test = y_test   
Ntoxicity = 3  

ws_train, vs_train = ws_train, Xtr 
ws_val, vs_val = ws_val, Xval   
ws_test, vs_test = ws_test, Xte  


dataG_train = [ws_train, vs_train]
dataG_val = [ws_val, vs_val]              
dataG_test = [ws_test, vs_test]

In [22]:
print(type(labelsG_train))
print(type(labelsG_test))
print(type(dataG_train))
print(type(dataG_val))
print(type(dataG_test))


print(labelsG_train.shape)
print(labelsG_val.shape)
print(labelsG_test.shape)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'list'>
<class 'list'>
<class 'list'>
(234622,)
(49878,)
(50571,)


# Compute Coulomb matrix

In [23]:
def compute_coulomb_matrix_for_view(view):
    """
    Compute Coulomb matrix for a single view.
    
    Parameters:
    - view: numpy array of shape (29, 4) where each row is [atomic_number, x, y, z]
    
    Returns:
    - coulomb_matrix: numpy array of shape (29, 29)
    """
    n_atoms = view.shape[0]
    coulomb_matrix = np.zeros((n_atoms, n_atoms))
    
    for i in range(n_atoms):
        atomic_num_i = view[i, 0]
        if atomic_num_i == 0:
            continue
            
        coords_i = view[i, 1:4]
        
        for j in range(n_atoms):
            atomic_num_j = view[j, 0]
            if atomic_num_j == 0:  # Skip padding atoms
                continue
                
            coords_j = view[j, 1:4]
            
            if i == j:
                # Diagonal element: self-interaction
                coulomb_matrix[i, j] = 0.5 * atomic_num_i ** 2.4
            else:
                # Off-diagonal element: interaction between atoms i and j
                dist = np.linalg.norm(coords_i - coords_j)
                coulomb_matrix[i, j] = atomic_num_i * atomic_num_j / dist
                
    return coulomb_matrix

In [24]:
def precompute_coulomb_matrices_for_views(views_data):
    """
    Precompute Coulomb matrices for all views in the dataset.
    
    Parameters:
    - views_data: numpy array of shape (batch_size, num_views, num_atoms, 4)
    
    Returns:
    - coulomb_matrices: numpy array of shape (batch_size, num_views, num_atoms, num_atoms)
    """
    batch_size, num_views, num_atoms, _ = views_data.shape
    coulomb_matrices = np.zeros((batch_size, num_views, num_atoms, num_atoms))
    
    for batch_idx in range(batch_size):
        for view_idx in range(num_views):
            view = views_data[batch_idx, view_idx]
            coulomb_matrices[batch_idx, view_idx] = compute_coulomb_matrix_for_view(view)
                
    return coulomb_matrices

# Load Coulomb matrices

In [26]:
coulomb_train = np.load("coulomb_train.npy", mmap_mode='r')
coulomb_val   = np.load("coulomb_val.npy",   mmap_mode='r')
coulomb_test  = np.load("coulomb_test.npy",  mmap_mode='r')



In [27]:
print(coulomb_train.shape)
print(coulomb_val.shape)
print(coulomb_test.shape)

(234622, 29, 29, 29)
(49878, 29, 29, 29)
(50571, 29, 29, 29)


In [28]:
coulomb_train[0][0][0]

memmap([2539.41283766,  107.98894501,   72.79267883,   71.78925323,
          71.77416992,   53.12495804,   48.20129013,   46.40313339,
          43.5627861 ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ])

In [29]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import tensorflow as tf
from rdkit import Chem
import itertools
import numpy as np
import pandas as pd
import os, datetime

# Transformer layer

In [ ]:
from tensorflow.keras.layers import Layer, LayerNormalization, Dense

class AtomAttention(Layer):
    
    def __init__(self, d_k=32, d_v=16, **kwargs):
        """
        Single-head atom-atom self-attention with optional Coulomb bias.
        d_k = dimension of query/key
        d_v = dimension of value
        """
        super(AtomAttention, self).__init__(**kwargs)
        self.d_k = d_k
        self.d_v = d_v
        self.norm = LayerNormalization(axis=-1, epsilon=1e-6)
        self.alpha = None  # scaling for Coulomb
        
    def build(self, input_shape):
        # input_shape: (batch, views, atoms, features)
        self.batch_size  = input_shape[0]
        self.num_views   = input_shape[1]
        self.num_atoms   = input_shape[2]
        self.feature_dim = input_shape[3]
        
        self.W_q = self.add_weight(
            name='W_q',
            shape=(self.feature_dim, self.d_k),
            initializer='glorot_uniform',
            trainable=True
        )
        self.W_k = self.add_weight(
            name='W_k',
            shape=(self.feature_dim, self.d_k),
            initializer='glorot_uniform',
            trainable=True
        )
        self.W_v = self.add_weight(
            name='W_v',
            shape=(self.feature_dim, self.d_v),
            initializer='glorot_uniform',
            trainable=True
        )
        self.W_o = self.add_weight(
            name='W_o',
            shape=(self.d_v, self.feature_dim),
            initializer='glorot_uniform',
            trainable=True
        )
        
        self.alpha = self.add_weight(
            name='alpha',
            shape=(),
            initializer=tf.keras.initializers.Constant(0.1),
            trainable=True
        )
        
    def call(self, inputs, coulomb_matrix=None):
        """
        inputs:  (batch, views, atoms, features)
        coulomb_matrix: (batch, views, atoms, atoms) or None
        output:  (batch, views, atoms, features)
        """
        
        reshaped_input = tf.reshape(inputs, [-1, self.num_atoms, self.feature_dim])
        
        # Q, K, V
        Q = tf.matmul(reshaped_input, self.W_q)  # (batch*views, atoms, d_k)
        K = tf.matmul(reshaped_input, self.W_k)  # (batch*views, atoms, d_k)
        V = tf.matmul(reshaped_input, self.W_v)  # (batch*views, atoms, d_v)
        
        # Raw attention scores
        scores = tf.matmul(Q, K, transpose_b=True)  # (batch*views, atoms, atoms)
        scaled_scores = scores / tf.math.sqrt(tf.cast(self.d_k, tf.float32))
        
        #
        if coulomb_matrix is not None:
            # coulomb_matrix: (batch, views, atoms, atoms) -> (batch*views, atoms, atoms)
            coulomb_reshaped = tf.reshape(
                coulomb_matrix,
                [-1, self.num_atoms, self.num_atoms]
            )
            # make sure types match
            coulomb_reshaped = tf.cast(coulomb_reshaped, scaled_scores.dtype)
            scaled_scores = scaled_scores + self.alpha * coulomb_reshaped
        
        # Mask zero-padded atoms (as in your original code)
        input_sums = tf.reduce_sum(tf.abs(inputs), axis=[0, 1, 3])  # (atoms,)
        mask_atoms = tf.equal(input_sums, 0.0)  # True for padded atoms
        
        mask_rows = tf.expand_dims(mask_atoms, 1)  # (atoms, 1)
        mask_cols = tf.expand_dims(mask_atoms, 0)  # (1, atoms)
        mask = tf.logical_or(mask_rows, mask_cols)  # (atoms, atoms)
        
        batch_views = tf.shape(scaled_scores)[0]
        mask = tf.tile(tf.expand_dims(mask, 0), [batch_views, 1, 1])  # (batch*views, atoms, atoms)
        
        # Softmax
        attn_weight = tf.nn.softmax(scaled_scores, axis=-1)
        
        # Apply mask AFTER softmax: zero out masked positions
        attn_weight = tf.where(mask, 0.0, attn_weight)
        
        # Weighted sum of values
        attn_output = tf.matmul(attn_weight, V)  # (batch*views, atoms, d_v)
        
        # Project back to feature space
        project_output = tf.matmul(attn_output, self.W_o)  # (batch*views, atoms, features)
        
        # Residual + LayerNorm
        skip_output = project_output + reshaped_input
        normed = self.norm(skip_output)
        
        # Reshape back to (batch, views, atoms, features)
        batch_size = tf.shape(inputs)[0]
        final_output = tf.reshape(
            normed,
            [batch_size, self.num_views, self.num_atoms, self.feature_dim]
        )
        return final_output


class FeedForward(Layer):
    """Position-wise feed-forward net with residual + LayerNorm."""
    def __init__(self, d_model=20, d_ff=64, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fc1 = Dense(d_ff, activation='relu', name='ffn_fc1')
        self.fc2 = Dense(d_model, name='ffn_fc2')
        self.norm = LayerNormalization(axis=-1, epsilon=1e-6, name='ffn_norm')

    def call(self, x):
        y = self.fc1(x)
        y = self.fc2(y)
        return self.norm(x + y)


class TransformerBlock(Layer):
    """
    Single Transformer encoder block: AtomAttention (+Coulomb) + FeedForward.
    """
    def __init__(self, d_model=20, d_k=32, d_v=16, d_ff=64, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.attn = AtomAttention(d_k=d_k, d_v=d_v, name='atom_attention')
        self.ffn = FeedForward(d_model=d_model, d_ff=d_ff, name='ffn')

    def call(self, x, coulomb_matrix=None):
        x = self.attn(x, coulomb_matrix=coulomb_matrix)
        x = self.ffn(x)
        return x


In [31]:
# generic dense NN
def multiDense(Nin,Nout,Nhidden,widthhidden=None,kernel_regularizer=None):
    """Construct a basic NN with some dense layers.
    
    :parameter Nin: The number of inputs
    :type Nin: int
    :parameter Nout: The number of outputs
    :type Nout: int
    :parameter Nhidden: The number of hidden layers.
    :type Nhidden: int
    :parameter widthhidden: The width of each hidden layer.
        If left at None, Nin + Nout will be used.
    :parameter kernel_regularizer: the regularizer to use, such as regularizers.l2(0.001)
    :type kernel_regularizer: tensorflow.keras.regularizers.xxx
    :returns: The NN model
    :rtype: keras.Model
    
    """
    if widthhidden is None:
        widthhidden = Nin + Nout
    x = inputs = keras.Input(shape=(Nin,), name='multiDense_input')
    if kernel_regularizer is not None:
        print("Using regularization")
    for i in range(Nhidden):
        x = layers.Dense(widthhidden, activation='relu', kernel_regularizer=kernel_regularizer,name='dense'+str(i))(x)
#        x = layers.Dense(widthhidden, name='dense'+str(i))(x)
#        x = tf.nn.leaky_relu(x, alpha=0.05)
#    outputs = layers.Dense(Nout, activation='linear',name='multiDense_output')(x)
    outputs = layers.Dense(Nout,name='multiDense_output')(x)
    #outputs = tf.nn.leaky_relu(outputs, alpha=0.05)
    return keras.Model(inputs=inputs, outputs=outputs)#, name='multiDense')
if 1:
    # manual check of multiDense
    mmd = multiDense(580,3,10,583)
    mmd.summary()
    # used to do the weighted sum over views

def parallelwrapper(Nparallel,basemodel,insteadmax=False):
    """Construct a model that applies a basemodel multiple times and take a weighted sum (or max) of the result.
    
    :parameter Nparallel: The number of times to apply in parallel
    :type Nparallel: int
    :parameter basemodel: a keras.Model inferred to have Nin inputs and Nout outputs.
    :type basemodel: a keras.Model
    :parameter insteadmax: If True, take the max of the results of the basemodel instead of the weighted sum.
        For compatibility, the model is still constructed with weights as inputs, but it ignores them.
    :type insteadmax: Boolean
    :returns: model with inputs shape [(?,Nparallel),(?,Nin,Nparallel)] and outputs shape (?,Nout).
        The first input is the scalar weights in the sum.
    :rtype: keras.Model
    
    Note: We could do a max over the parallel applications instead of or in addition to the weighted sum.
    
    """
    # infer shape of basemodel inputs and outputs
    Nin =  basemodel.inputs[0].shape[1]
    Nout =  basemodel.outputs[0].shape[1]
    
    # Apply basemodel Nparallel times in parallel
    # create main input (?,Nparallel,Nin) 
    parallel_inputs = keras.Input(shape=(Nparallel,Nin), name='parallelwrapper_input0')
    # apply base NN to each parallel slice; outputs (?,Nparallel,Nout)
    if False:
        # original version, stopped working at some tensorflow update
        xb = basemodel(parallel_inputs) # worked in earlier tensorflow
        #xb = tf.map_fn(basemodel,parallel_inputs) # another version that fails
    else:
        # newer version, works but makes summary and graphing cumbersome
        # unstack in the Nparallel directio
        parallel_inputsunstacked = tf.keras.ops.unstack(parallel_inputs, Nparallel, 1)
        # apply base NN to each 
        xbunstacked = [basemodel(x) for x in parallel_inputsunstacked]
        # re-stack
        xb = tf.keras.ops.stack(xbunstacked,axis=1)
    
    # create input scalars for weighted sun (?,Nparallel)
    weight_inputs = keras.Input(shape=(Nparallel,), name='parallelScalars')
    if insteadmax:
        # take max over the Nparallel direction to get (?,1,Nout)
        out = layers.MaxPool1D(pool_size=Nparallel)(xb)
        # reshape to (?,Nout)
        out = layers.Reshape((Nout,))(out)
    else:
        # do a weighted sum over the Nparallel direction to get (?,Nout)
        out = layers.Dot((-2,-1))([xb,weight_inputs])
    
    return keras.Model(inputs=[weight_inputs,parallel_inputs], outputs=out, name='parallelwrapper')
if 1:
    # manual check
    mmd = multiDense(580,3,10,583)
    mpw = parallelwrapper(29,mmd,insteadmax=0)
    mpw.summary()
    # make models


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ multiDense_input (InputLayer)   │ (None, 580)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense0 (Dense)                  │ (None, 583)            │       338,723 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense3 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense4 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense5 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense6 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense7 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense8 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense9 (Dense)                  │ (None, 583)            │       340,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multiDense_output (Dense)       │ (None, 3)              │         1,752 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,404,723 (12.99 MB)

 Trainable params: 3,404,723 (12.99 MB)

 Non-trainable params: 0 (0.00 B)

Model: "parallelwrapper"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ parallelwrapper_in… │ (None, 29, 580)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ unstack (Unstack)   │ [(None, 580),     │          0 │ parallelwrapper_… │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580),      │            │                   │
│                     │ (None, 580)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 3)         │  3,404,723 │ unstack[0][0],    │
│ (Functional)        │                   │            │ unstack[0][1],    │
│                     │                   │            │ unstack[0][2],    │
│                     │                   │            │ unstack[0][3],    │
│                     │                   │            │ unstack[0][4],    │
│                     │                   │            │ unstack[0][5],    │
│                     │                   │            │ unstack[0][6],    │
│                     │                   │            │ unstack[0][7],    │
│                     │                   │            │ unstack[0][8],    │
│                     │                   │            │ unstack[0][9],    │
│                     │                   │            │ unstack[0][10],   │
│                     │                   │            │ unstack[0][11],   │
│                     │                   │            │ unstack[0][12],   │
│                     │                   │            │ unstack[0][13],   │
│                     │                   │            │ unstack[0][14],   │
│                     │                   │            │ unstack[0][15], 

 Total params: 3,404,723 (12.99 MB)

 Trainable params: 3,404,723 (12.99 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:

def init_generator_with_transformer(
    data, labels, baselayers, Nfeatures, endlayers, Nviews,
    num_blocks=4, d_k=32, d_v=16, d_ff=64,
    base_regularizer=None, end_regularizer=None
):
    """
    data: [ws, vs, coulomb] for the *whole dataset* (used to infer shapes)
    """
    # data[1] is vs: (Nmol, Nviews, Natoms, d_model)
    Natoms  = data[1].shape[2]
    d_model = data[1].shape[3]
    flattened_dim = Natoms * d_model
    
    # Inputs
    weight_input = keras.Input(shape=(Nviews,), name='weight_input')              # (batch, Nviews)
    atom_input   = keras.Input(shape=(Nviews, Natoms, d_model), name='atom_input')# (batch, Nviews, Natoms, d_model)
    coulomb_input = keras.Input(
        shape=(Nviews, Natoms, Natoms),
        name='coulomb_input'
    )  # (batch, Nviews, Natoms, Natoms)
    
    # Transformer blocks with Coulomb
    x = atom_input
    for i in range(num_blocks):
        x = TransformerBlock(
            d_model=d_model,
            d_k=d_k,
            d_v=d_v,
            d_ff=d_ff,
            name=f'transformer_block_{i}'
        )(x, coulomb_matrix=coulomb_input)
    
    # Flatten per view
    x = layers.Reshape((Nviews, flattened_dim), name='flatten_views')(x)
    
    # View-wise NN + weighted sum over views
    Gbase = multiDense(flattened_dim, Nfeatures, baselayers, kernel_regularizer=base_regularizer)
    Gpw   = parallelwrapper(Nviews, Gbase, insteadmax=False)
    x = Gpw([weight_input, x])  # (batch, Nfeatures)
    
    # Final regressor
    Gft = multiDense(Nfeatures, 1, endlayers, kernel_regularizer=end_regularizer)
    output = Gft(x)  # (batch, 1)
    
    generator = keras.Model(
        inputs=[weight_input, atom_input, coulomb_input],
        outputs=output,
        name='generator_with_transformer_coulomb'
    )
    return generator

In [33]:
def get_enantiomer_pairs(df):
    """
    Extract enantiomer pairs from dataframe based on SMILES_nostereo.
    """
    import numpy as np
    pairs = []
    smiles_groups = df.groupby('SMILES_nostereo')
    
    for smiles, group in smiles_groups:
        if len(group) >= 2:
            score_groups = group.groupby('top_score')
            enantiomer_data = []
            for score, score_group in score_groups:
                enantiomer_data.append({
                    'conformers': score_group.index.tolist(),
                    'top_score': score,
                    'size': len(score_group)
                })
            if len(enantiomer_data) == 2:
                if np.random.random() > 0.5:
                    pairs.append({
                        'enantiomer1_conformers': enantiomer_data[0]['conformers'],
                        'enantiomer2_conformers': enantiomer_data[1]['conformers'],
                        'enantiomer1_score': enantiomer_data[0]['top_score'],
                        'enantiomer2_score': enantiomer_data[1]['top_score'],
                        'smiles_nostereo': smiles
                    })
                else:
                    pairs.append({
                        'enantiomer1_conformers': enantiomer_data[1]['conformers'],
                        'enantiomer2_conformers': enantiomer_data[0]['conformers'],
                        'enantiomer1_score': enantiomer_data[1]['top_score'],
                        'enantiomer2_score': enantiomer_data[0]['top_score'],
                        'smiles_nostereo': smiles
                    })
    print(f"Found {len(pairs)} enantiomer pairs")
    return pairs


def create_enantiomer_batch(pairs, data, labels, batch_size=32):
    """
    data: [ws, vs, coulomb]
    """
    batch_ws    = []
    batch_vs    = []
    batch_coul  = []
    batch_scores = []
    
    ws, vs, coulomb = data
    n_pairs = len(pairs)
    
    if n_pairs < batch_size // 2:
        selected_pairs = np.random.choice(pairs, size=batch_size//2, replace=True)
    else:
        selected_pairs = np.random.choice(pairs, size=batch_size//2, replace=False)
    
    for pair in selected_pairs:
        conf1_idx = np.random.choice(pair['enantiomer1_conformers'])
        conf2_idx = np.random.choice(pair['enantiomer2_conformers'])
        
        batch_ws.extend([ws[conf1_idx], ws[conf2_idx]])
        batch_vs.extend([vs[conf1_idx], vs[conf2_idx]])
        batch_coul.extend([coulomb[conf1_idx], coulomb[conf2_idx]])
        batch_scores.extend([labels[conf1_idx], labels[conf2_idx]])
    
    return (
        np.array(batch_ws),
        np.array(batch_vs),
        np.array(batch_coul),
        np.array(batch_scores).reshape(-1, 1)
    )


def train_with_enantiomer_sampling(
    generator,
    dataG_train, labelsG_train, train_df,
    dataG_val,   labelsG_val,   val_df,
    epochs=10, batch_size=32
):
    """
    dataG_*: [ws, vs, coulomb]
    """
    enantiomer_pairs_train = get_enantiomer_pairs(train_df)
    if not enantiomer_pairs_train:
        raise ValueError("No enantiomer pairs found in training data!")
    
    best_val_accuracy = 0.0
    best_weights = None
    patience = 15
    patience_counter = 0
    
    num_batches_per_epoch = max(1, len(enantiomer_pairs_train) // (batch_size // 2))
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}", end=' - ')
        epoch_loss = 0.0
        
        for batch_idx in range(num_batches_per_epoch):
            batch_ws, batch_vs, batch_coul, batch_scores = create_enantiomer_batch(
                enantiomer_pairs_train, dataG_train, labelsG_train, batch_size
            )
            loss = generator.train_on_batch(
                [batch_ws, batch_vs, batch_coul],
                batch_scores
            )
            loss = float(loss[0] if isinstance(loss, (list, tuple)) else loss)
            epoch_loss += loss
        
        avg_loss = epoch_loss / num_batches_per_epoch
        print(f"Loss: {avg_loss:.4f}", end=' - ')
        
        val_accuracy = evaluate_ranking_accuracy(generator, dataG_val, labelsG_val, val_df)
        print(f"Val Acc: {val_accuracy:.4f}")
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_weights = generator.get_weights()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    if best_weights is not None:
        generator.set_weights(best_weights)
    return generator, best_val_accuracy


def evaluate_ranking_accuracy(model, data, labels, df, batch_size=32):
    """
    data: [ws, vs, coulomb]
    """
    ws, vs, coulomb = data
    all_predictions = []
    
    for i in range(0, len(ws), batch_size):
        batch_ws   = ws[i:i+batch_size]
        batch_vs   = vs[i:i+batch_size]
        batch_coul = coulomb[i:i+batch_size]
        batch_preds = model.predict(
            [batch_ws, batch_vs, batch_coul],
            verbose=0
        )
        all_predictions.extend(batch_preds.flatten())
    
    all_predictions = np.array(all_predictions)
    
    pairs = get_enantiomer_pairs(df)
    correct_rankings = 0
    total_pairs = len(pairs)
    
    for pair in pairs:
        preds1 = [all_predictions[idx] for idx in pair['enantiomer1_conformers']]
        preds2 = [all_predictions[idx] for idx in pair['enantiomer2_conformers']]
        avg_pred1 = np.mean(preds1)
        avg_pred2 = np.mean(preds2)
        true_score1 = pair['enantiomer1_score']
        true_score2 = pair['enantiomer2_score']
        pred_ranking = avg_pred1 < avg_pred2
        true_ranking = true_score1 < true_score2
        if pred_ranking == true_ranking:
            correct_rankings += 1
    
    accuracy = correct_rankings / total_pairs if total_pairs > 0 else 0.0
    return accuracy

In [ ]:

dataG_train = [
    ws_train.astype('float32'),
    Xtr.astype('float32'),
    coulomb_train.astype('float32')
]

dataG_val = [
    ws_val.astype('float32'),
    Xval.astype('float32'),
    coulomb_val.astype('float32')
]

dataG_test = [
    ws_test.astype('float32'),
    Xte.astype('float32'),
    coulomb_test.astype('float32')
]
# Build model
baselayers = 4
base_reg   = 0
Nfeatures  = 3
endlayers  = 2
end_reg    = 0
Nviews     = 29
# Print hyperparameters used
print("\n================ Hyperparameters ================")
print(f"Base layers:        {baselayers}")
print(f"Base regularizer:   {base_reg}")
print(f"Nfeatures:          {Nfeatures}")
print(f"End layers:         {endlayers}")
print(f"End regularizer:    {end_reg}")
print(f"Nviews:             {Nviews}")
print("Transformer params:")
print(f"  num_blocks:       4")
print(f"  d_k:              32")
print(f"  d_v:              20")
print(f"  d_ff:             64")
print("=================================================\n")


base_regularizer = regularizers.l2(base_reg) if base_reg else None
end_regularizer  = regularizers.l2(end_reg)  if end_reg  else None

generator = init_generator_with_transformer(
    dataG_train, labelsG_train,
    baselayers, Nfeatures, endlayers, Nviews,
    num_blocks=4, d_k=32, d_v=20, d_ff=64,
    base_regularizer=base_regularizer,
    end_regularizer=end_regularizer
)
generator.compile(optimizer='adam', loss='mse')

print("Starting training with enantiomer pair sampling...")
print(f"Training data:   {len(train_df)} conformers")
print(f"Validation data: {len(val_df)} conformers")
print(f"Test data:       {len(test_df)} conformers")

trained_generator, best_val_accuracy = train_with_enantiomer_sampling(
    generator, dataG_train, labelsG_train, train_df,
    dataG_val,   labelsG_val,   val_df,
    epochs=50, batch_size=32
)

print(f"Best validation accuracy: {best_val_accuracy:.4f}")

print("\nEvaluating on test set...")
test_accuracy = evaluate_ranking_accuracy(trained_generator, dataG_test, labelsG_test, test_df)
print(f"Final test ranking accuracy: {test_accuracy:.4f}")



================ Hyperparameters ================
Base layers:        4
Base regularizer:   0
Nfeatures:          3
End layers:         2
End regularizer:    0
Nviews:             29
Transformer params:
  num_blocks:       4
  d_k:              32
  d_v:              20
  d_ff:             64

Starting training with enantiomer pair sampling...
Training data:   234622 conformers
Validation data: 49878 conformers
Test data:       50571 conformers
Found 24192 enantiomer pairs
Epoch 1/50 - Loss: 2.4909 - Found 5184 enantiomer pairs
Val Acc: 0.4973
Epoch 2/50 - Loss: 1.1345 - Found 5184 enantiomer pairs
Val Acc: 0.4996
Epoch 3/50 - Loss: 1.0469 - Found 5184 enantiomer pairs
Val Acc: 0.4983
Epoch 4/50 - Loss: 1.0116 - Found 5184 enantiomer pairs
Val Acc: 0.4878
Epoch 5/50 - Loss: 0.9903 - Found 5184 enantiomer pairs
Val Acc: 0.5015
Epoch 6/50 - Loss: 0.9759 - Found 5184 enantiomer pairs
Val Acc: 0.5002
Epoch 7/50 - Loss: 0.9673 - Found 5184 enantiomer pairs
Val Acc: 0.4967
Epoch 8/50 - Loss